In [2]:
#load data file
import os
import pandas as pd
import traceback
from tkinter import Tk, filedialog

# 選取多個資料檔案

def get_file_paths():
    root = Tk()
    root.attributes('-topmost', True)  # 將主視窗設置為最上層
    root.withdraw()

    file_paths = []

    while True:
        files = filedialog.askopenfilenames(parent=root, title='選擇資料檔案', filetypes=[('CSV files', '*.csv')])
        if not files:
            break
        file_paths.extend(files)

    return file_paths

data_file_paths = get_file_paths()

all_raw_df = {}
data_file_name = {}


# 循環處理每個選取的檔案
for index, data_file_dir in enumerate(data_file_paths):
    try:
        data_file_name[index] = os.path.basename(data_file_dir)

        df = pd.read_csv(data_file_dir)
        all_raw_df[index] = df

    except Exception as e:
        print("格式錯誤檔案:" + data_file_name[index])
        traceback.print_exc()

C:\Users\Meng-Xuan Liu\AppData\Local\Temp\ipykernel_25300\1186686303.py:35: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file_dir)


In [5]:
excel_writer = pd.ExcelWriter(r"G:\works\Data_archive\PR_test\Cutoff1min_30min_summary.xlsx")
result = pd.DataFrame()

for index, df in enumerate(all_raw_df.values()):

    # 找到時間或Last_press大於1800000的行
    df = df.loc[(df['Time'] > 900000)].copy()
    filtered_df = df.loc[(df['Time'] > 1800000) | (df['Last_press'] > 60000)].copy()
    filtered_df.loc[:, 'ID'] = data_file_name[index]

    # 選擇"Lever_Press"和"Reward"列的第一行
    row_to_add = pd.DataFrame(filtered_df[['Lever_Press', 'Reward', 'ID']].iloc[0]).transpose()
    result = pd.concat([result, row_to_add], ignore_index=True)
    # 假設result是一個DataFrame
    columns = ['ID'] + [col for col in result.columns if col != 'ID']
    result = result.reindex(columns=columns)

result.to_excel(excel_writer, index=False)
excel_writer.close()